In [1]:
import character
import simulator

raw_data = simulator.simulate_combat(
    character.generate_archers(),
    target_ac=13,
    iterations=100
)

In [2]:
import pandas as pd

df = pd.DataFrame(raw_data)
display(df)

,iteration_number,round_number,name,level,round_damage
0,0,1,Ranger Archery,1,10
1,0,1,Ranger TwoWeaponFighting,1,13
2,0,1,Ranger Archery,2,0
3,0,1,Ranger TwoWeaponFighting,2,7
4,0,1,Ranger Archery,3,9
...,...,...,...,...,...
11995,99,5,Ranger TwoWeaponFighting,10,25
11996,99,5,Ranger Archery,11,23
11997,99,5,Ranger TwoWeaponFighting,11,23
11998,99,5,Ranger Archery,12,23


In [4]:
df_per_round_stats = df.groupby(['iteration_number', 'level', 'name']).agg(
    # mean_round_damage=('round_damage', 'mean'),
    median_round_damage=('round_damage', 'median'),
    # max_round_damage=('round_damage', 'max'),
    percentile_10_round_damage=('round_damage', lambda x: x.quantile(0.1)),
    percentile_90_round_damage=('round_damage', lambda x: x.quantile(0.9))
).reset_index()

df_per_round_stats


,iteration_number,level,name,median_round_damage,percentile_10_round_damage,percentile_90_round_damage
0,0,1,Ranger Archery,10.0,4.0,18.0
1,0,1,Ranger TwoWeaponFighting,13.0,2.8,19.6
2,0,2,Ranger Archery,8.0,0.0,12.6
3,0,2,Ranger TwoWeaponFighting,10.0,6.4,14.0
4,0,3,Ranger Archery,8.0,2.4,9.6
...,...,...,...,...,...,...
2395,99,10,Ranger TwoWeaponFighting,26.0,25.0,28.0
2396,99,11,Ranger Archery,27.0,23.4,32.8
2397,99,11,Ranger TwoWeaponFighting,29.0,24.6,30.8
2398,99,12,Ranger Archery,23.0,18.2,27.8


In [5]:
df_overall = df_per_round_stats.groupby(['level', 'name']).agg(
    median_DPR=('median_round_damage', 'mean'),
    percentile_10_DPR=('percentile_10_round_damage', 'mean'),
    percentile_90_DPR=('percentile_90_round_damage', 'mean')
).reset_index()

df_overall

,level,name,median_DPR,percentile_10_DPR,percentile_90_DPR
0,1,Ranger Archery,8.53,3.958,12.814
1,1,Ranger TwoWeaponFighting,10.50,5.328,15.970
2,2,Ranger Archery,7.92,3.746,11.692
3,2,Ranger TwoWeaponFighting,10.60,5.222,15.438
4,3,Ranger Archery,10.10,4.856,13.322
5,3,Ranger TwoWeaponFighting,13.55,7.132,18.198
6,4,Ranger Archery,9.58,4.640,13.314
7,4,Ranger TwoWeaponFighting,13.31,7.264,17.922
8,5,Ranger Archery,19.78,14.104,24.860
9,5,Ranger TwoWeaponFighting,21.26,14.350,26.912


In [8]:
import plotly.express as px

fig = px.line(df_overall,
              x='level',
              y='median_DPR',
              color='name')
fig.show()